# Downloading and Processing ECMWF hindcasts 

## Check for possibility of a readiness trigger with a longer lead time

The current forecast (Meteo France La Reunion) provides limited lead time and we should explore if we could use ECMWF to have a readiness signal. I would basically check some basic perfomance metrics vs lead time and decide with partners what is acceptable.

In [ ]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [ ]:

import os
from datetime import datetime
from pathlib import Path
import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

import re
import xml.etree.ElementTree as ET
from src.datasources import codab, rsmc, helpers
from src.constants import *

In [ ]:
save_dir = (
    Path(AA_DATA_DIR) / "public" / "exploration" / "moz" / "ecmwf_hindcast"
)
ADMS = ADMS
adm1_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]
ibtracs_path = adm1_path = (
    Path(AA_DATA_DIR) / "public" / "raw" / "glb" / "ibtracs"
)
points_path = Path(
    ibtracs_path
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)
gdf_points = gpd.read_file(points_path)
gdf_points["Date"] = [
    datetime.strptime(dt, "%Y-%m-%d %H:%M:%S").date()
    for dt in gdf_points["ISO_TIME"]
]

In [4]:
gdf_points_moz = gpd.sjoin(gdf_sel, gdf_points, predicate="intersects")

In [5]:
gdf_points_moz_2006 = gdf_points_moz[gdf_points_moz["year"] >= 2006]

In [16]:
# loading all actual cyclone tracks
cyclone_tracks = pd.read_csv(
    rsmc.DATA_DIR
    / "private"
    / "raw"
    / "moz"
    / "rsmc"
    / "data_cyclone_SWIO_19851986_to_20222023.csv"
)
cyclone_tracks["Lat"] = cyclone_tracks["Lat"].apply(
    lambda x: -x if x > 0 else x
)
cyclone_tracks["geometry"] = cyclone_tracks.apply(
    lambda row: Point(row["Lon"], row["Lat"]), axis=1
)
cyclone_tracks_gdf = gpd.GeoDataFrame(
    cyclone_tracks, geometry="geometry", crs="EPSG:4326"
)
cyclone_tracks_sel = gpd.sjoin(
    cyclone_tracks_gdf, gdf_sel, how="inner", predicate="intersects"
)
cyclone_tracks_sel[cyclone_tracks_sel["Year"] >= 2006]["Name"].unique()

array(['720052006', 'FAVIO', 'JAYA', 'JOKWE', 'IZILDA', 'DANDO', 'IRINA',
       'HARUNA', 'DELIWE', '920132014', 'GUITO', 'HELLEN', 'CHEDZA',
       'DINEO', '420172018', 'DESMOND', 'IDAI', 'KENNETH', 'CHALANE',
       'ELOISE', 'GUAMBE', 'ANA', 'DUMAKO', 'GOMBE', 'JASMINE', 'FREDDY'],
      dtype=object)

### Getting dates for cyclones

In [6]:
min_max_dates = (
    gdf_points[gdf_points["SID"].isin(gdf_points_moz_2006["SID"])]
    .groupby(["SID", "NAME"])["Date"]
    .agg(["min", "max"])
    .reset_index()
)

In [7]:
min_max_dates

,SID,NAME,min,max
0,2007043S11071,FAVIO,2007-02-11,2007-02-23
1,2008062S10064,JOKWE,2008-03-02,2008-03-16
2,2009082S16039,IZILDA,2009-03-22,2009-03-29
3,2012010S24049,DANDO,2012-01-10,2012-01-18
4,2012056S13057,IRINA,2012-02-25,2012-03-12
5,2013046S20042,HARUNA,2013-02-14,2013-02-28
6,2014015S16043,DELIWE,2014-01-14,2014-01-22
7,2014048S16039,GUITO,2014-02-17,2014-02-24
8,2014086S10041,HELLEN,2014-03-26,2014-04-05
9,2015013S18038,CHEDZA,2015-01-13,2015-01-22


### Downloading Hindcasts

In [ ]:
base_url = "https://data.rda.ucar.edu/ds330.3/ecmf/"
times = ["000000", "120000"]
date_lists = []
for index, row in min_max_dates.iterrows():
    min_date = row["min"]
    max_date = row["max"]
    date_list = pd.date_range(start=min_date, end=max_date).tolist()
    for date in date_list:
        date = date.strftime("%Y%m%d")
        year = date[0:4]
        for time in times:
            server = "test"
            if date >= 20080801:
                server = "prod"
            filename = f"z_tigge_c_ecmf_{date}{time}_ifs_glob_{server}_all_glo.xml"
            filename_url = f"{base_url}{year}/{date}/{filename}"
            print("Downloading", filename)
            req = requests.get(filename_url, allow_redirects=True)
            open(Path(save_dir) / "xml" / filename, "wb").write(req.content)

### Processing hindcasts

In [97]:
def xml2csv(filename):
    print(f"{filename}")
    try:
        tree = ET.parse(filename)
    except ET.ParseError:
        print("Error with file, skipping")
        return
    root = tree.getroot()

    prod_center = root.find("header/productionCenter").text
    baseTime = root.find("header/baseTime").text

    # Create one dictonary for each time point, and append it to a list
    for members in root.findall("data"):
        mtype = members.get("type")
        if mtype not in ["forecast", "ensembleForecast"]:
            continue
        for members2 in members.findall("disturbance"):
            cyclone_name = [
                name.text.lower().strip() for name in members2.findall("cycloneName")
            ]
            if not cyclone_name:
                continue
            cyclone_name = cyclone_name[0].lower()
            if cyclone_name not in list(min_max_dates["NAME"].str.lower()):
                continue
            print(f"Found cyclone {cyclone_name}")
            for members3 in members2.findall("fix"):
                tem_dic = {}
                tem_dic["mtype"] = [mtype]
                tem_dic["product"] = [re.sub("\\s+", " ", prod_center).strip().lower()]
                tem_dic["cyc_number"] = [
                    name.text for name in members2.findall("cycloneNumber")
                ]
                tem_dic["ensemble"] = [members.get("member")]
                tem_dic["speed"] = [
                    name.text
                    for name in members3.findall("cycloneData/maximumWind/speed")
                ]
                tem_dic["pressure"] = [
                    name.text
                    for name in members3.findall("cycloneData/minimumPressure/pressure")
                ]
                time = [name.text for name in members3.findall("validTime")]
                tem_dic["time"] = [
                    "/".join(time[0].split("T")[0].split("-"))
                    + ", "
                    + time[0].split("T")[1][:-1]
                ]
                tem_dic["lat"] = [name.text for name in members3.findall("latitude")]
                tem_dic["lon"] = [name.text for name in members3.findall("longitude")]
                tem_dic["lead_time"] = [members3.get("hour")]
                tem_dic["forecast_time"] = [
                    "/".join(baseTime.split("T")[0].split("-"))
                    + ", "
                    + baseTime.split("T")[1][:-1]
                ]
                tem_dic1 = dict(
                    [
                        (k, "".join(str(e).lower().strip() for e in v))
                        for k, v in tem_dic.items()
                    ]
                )
                # Save to CSV
                outfile = save_dir / f"csv/{cyclone_name}_all.csv"
                pd.DataFrame(tem_dic1, index=[0]).to_csv(
                    outfile,
                    mode="a",
                    header=not os.path.exists(outfile),
                    index=False,
                )

In [ ]:
filename_list = sorted(list(Path(save_dir / "xml").glob("*.xml")))
for filename in filename_list:
    xml2csv(filename)